### Inicio

In [40]:
from datetime import date
import os
os.environ["AWS_PROFILE"] = "men"
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [41]:
params = {
    "class_name":"Cocina",
    "target_audience_edu_level":"Preescolar",
    "grade":"no aplica"
}

### Retriever

In [42]:
from langchain_aws.retrievers.bedrock import AmazonKnowledgeBasesRetriever

In [43]:
kb_context_id = "BC3JS4SRQG"
kb_result_id = "MQPBML7IQ2"

# Base knowledge
retriever_context = AmazonKnowledgeBasesRetriever(
    knowledge_base_id = kb_context_id,
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
            'overrideSearchType': "SEMANTIC" # optional
        }
    },
)

retriever_result = AmazonKnowledgeBasesRetriever(
    knowledge_base_id = kb_result_id,
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
            'overrideSearchType': "SEMANTIC" # optional
        }
    },
)

In [44]:
def get_retriver_docs(docs):    
    context = "\n".join([doc.page_content for doc in docs])  # Iteramos de forma síncrona si 'docs' es una lista
    return context

In [45]:
async def get_retriver_context(params):
    docs = await retriever_context.ainvoke(params)
    return get_retriver_docs(docs)

In [46]:
async def get_retriver_result(params):
    docs = await retriever_result.ainvoke(params)
    return get_retriver_docs(docs)

### Prompts

In [47]:
from langchain.prompts import PromptTemplate

In [48]:
def get_relevant_info():
    with open("1_get_relevant_info.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
        input_variables=["class_name"],
        template=prompt_content,
    )



In [49]:
def get_base_prompt():
    with open("2_get_base_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()

    return PromptTemplate(
        input_variables=["target_audience_edu_level", "class_name", "grade", "date", "context", "learning_result"],
        template=prompt_content
    )

In [50]:
def get_preschool_able_prompt():
    with open("3_get_preschool_able_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["result_1"],
            template=prompt_content
        )

In [51]:
def get_key_concepts():
    with open("4_get_key_concepts.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["result_1", "target_audience_edu_level", "class_name", "grade", "context"],
            template=prompt_content
        )

In [52]:
def get_activities_prompt():
    with open("5_get_activities_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["result_1_2", "target_audience_edu_level", "class_name", "grade", "context", "dba"],
            template=prompt_content
        )

In [53]:
def get_exam_prompt():
    with open("6_get_exam_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["prompt"],
            template=prompt_content
        )

In [54]:
def get_coherence_prompt():
    with open("get_coherence_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
        input_variables=["result_1", "result_2", "result_3", "result_4", "target_audience_edu_level"],
        template=prompt_content,
    )

### Definir el LLM


In [55]:
from langchain_aws import BedrockLLM
from langchain_aws import ChatBedrock, ChatBedrockConverse

bedrock_model = "anthropic.claude-3-sonnet-20240229-v1:0"

llm_instance = ChatBedrockConverse(
    model=bedrock_model,
    temperature=0.02,
    max_tokens=None
)

### Parser y Filtro de Relevancia

In [56]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [57]:
relevance = get_relevant_info() | llm_instance | parser
relevance_params = {
    "class_name": params["class_name"]
}
relevante = relevance.invoke(relevance_params)


### Proceso Parte 1 :  Resultados de Aprendizaje

In [58]:
if relevante == 'NO':
  print("El tema que estás buscando no corresponde a un tema formativo, o actualmente no contamos con suficiente información para generar una clase sobre este asunto.")

else:
  prompt_template = get_base_prompt()
  classes = params["class_name"]
  context_query = f"{params['class_name']}, para grados {params['grade']}"
  context = get_retriver_context(context_query)

  dba_query = f"De los derechos basicos de aprendizaje dame la metodologia de enseñanza para la tematica de ({classes}) dirigida a estudiantes del grado {params['grade']}"
  dba = get_retriver_result(dba_query)

  learning_result_query = f"Dame información de los Estándares Básicos de Competencias y Orientaciones pedagógicas para la clase ({classes}) dirigida a estudiantes del grado {params['grade']}"
  learning_result = get_retriver_result(learning_result_query)

  chain = prompt_template | llm_instance | parser

  new_params = {
            "class_name": params["class_name"],
                "target_audience_edu_level":params["target_audience_edu_level"],
                "grade": params["grade"],
                "context": context,
                "dba": dba,
                "learning_result": learning_result,
                "date":  date.today()
            } 
  
  result_1 = chain.invoke(new_params)

  if params["target_audience_edu_level"] == "Preescolar":
      preschool_able = get_preschool_able_prompt() | llm_instance | parser
      preschool_able_params = {"result": result_1}
      result_1 = preschool_able.invoke(preschool_able_params)

C:\Users\nuvup\AppData\Local\Temp\ipykernel_9156\1491747051.py:18: RuntimeWarning: coroutine 'get_retriver_result' was never awaited
  new_params = {


### Proceso parte 2 : Conceptos clave

In [59]:
key_template = get_key_concepts()

chain_key_concepts = key_template | llm_instance | parser
key_params = {
    "result_1": result_1,
    "class_name": params["class_name"],
    "target_audience_edu_level":params["target_audience_edu_level"],
    "grade": params["grade"],
    "context": context
}

result_2 = chain_key_concepts.invoke(key_params)

if params["target_audience_edu_level"] == "Preescolar":
    result_2 = preschool_able.invoke({"result": result_2})

C:\Users\nuvup\AppData\Local\Temp\ipykernel_9156\2596700815.py:4: RuntimeWarning: coroutine 'get_retriver_context' was never awaited
  key_params = {


### Proceso parte 3 : Actividades de aprendizaje

In [60]:
result_1_2 = result_1 + "\n\n" + result_2
activities_template = get_activities_prompt()
chain_activities = activities_template | llm_instance | parser

activities_params = {
    "result_1_2": result_1_2,
    "class_name": params["class_name"],
    "target_audience_edu_level": params["target_audience_edu_level"],
    "grade": params["grade"],
    "context": context,
    "dba": dba
}

result_3 = chain_activities.invoke(activities_params)

if params["target_audience_edu_level"] == "Preescolar":
    result_3 = preschool_able.invoke({"result": result_3})

### Proceso parte 4 : Examen

In [61]:
exam_template = get_exam_prompt()
chain_exam = exam_template | llm_instance | parser
examn_params = {
    "prompt": result_1 + '\n\n' + result_2 + '\n\n' + result_3 
}

result_4 = chain_exam.invoke(examn_params)

if params["target_audience_edu_level"] == "Preescolar":
    result_4 = preschool_able.invoke({"result": result_4})

# Ouput

In [62]:
print(result_1)
print()
print(result_2)
print()
print(result_3)
print()
print(result_4)

# ¡Cocinemos con Mamá y Papá!

Fecha de creación: 2025-01-22
Nivel educativo: Preescolar

Resultados de aprendizaje:

1. Conocer los alimentos que comemos en casa.
2. Practicar nuestras manitas para cocinar.
3. Aprender a comer comidas ricas y saludables.
4. Jugar y divertirnos cocinando juntos.

## Conceptos clave:

- **Comida**: Lo que comemos para estar sanos y fuertes.
- **Cocinar**: Preparar la comida con ayuda de un adulto.
- **Ingredientes**: Las cosas que necesitamos para hacer la comida.
- **Nutrientes**: Las cosas buenas que hay en la comida y nos ayudan a crecer.
- **Receta**: Las instrucciones para hacer una comida rica.
- **Saludable**: Algo que nos hace bien al cuerpo.
- **Utensilios**: Las herramientas que usamos para cocinar.

## Actividades divertidas

- Jugaremos a ser cocineros: Usaremos delantales y gorros de chef para sentirnos como verdaderos cocineros.
- Escucharemos cuentos y canciones sobre comidas: Nos divertiremos aprendiendo sobre los alimentos con historias